# DEPRECATED

# MISTRAL 1st METHOD - Doesn't work

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, StoppingCriteria, StoppingCriteriaList
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset, DatasetDict, Dataset, load_from_disk
from trl import SFTTrainer
import random

In [2]:
data_modified = "./en-ja_bis_2.tsv"

file_path_test = "./en-ja-test.txt"

cache_dir = "D:\\.cache"

In [3]:
dataset = load_dataset("text", data_files=file_path_test)


# def preprocess_func(sample):
#   eng_sent, jap_sent = sample.split('\t')
#   return {'text': "<s>[INST] Translate from English to Japanese: " + eng_sent + " [INST] " + jap_sent + " </s>"}


# dataset = dataset.map(preprocess_func)

def load_txt_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        data = {'id': [], 'en': [], 'jp': [], 'text': []}
        for idx, line in enumerate(lines):
            eng_sent, jap_sent = line.strip().split('\t')
            data['id'].append(idx)
            data['en'].append(eng_sent)
            data['jp'].append(jap_sent)
            data['text'].append("<s>[INST] Translate from English to Japanese: " + eng_sent + " [INST] " + jap_sent + " </s>")
        return Dataset.from_dict(data)


def split_train_test(dataset, test_ratio=0.2, seed=0, delete_after=False):
    random.seed(seed)
    n = len(dataset)
    index = list(range(n))
    random.shuffle(index)
    test_size = int(n * test_ratio)
    training_set = dataset.select(index[test_size:])
    test_set = dataset.select(index[:test_size])

    if delete_after:
        del dataset
    
    return training_set, test_set




dataset = load_txt_data(file_path_test)

train_data, test_data = split_train_test(dataset, test_ratio= 0.2, seed= 0, delete_after=True)

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
dataset = load_dataset('csv', data_files=data_modified, delimiter='\t', cache_dir=cache_dir)

train_data = dataset["train"].train_test_split(test_size= 0.2, seed= 0)
test_data = train_data["test"].train_test_split(test_size=0.5, seed=0)

def filter_non_empty(line):
    return all([line["en"] is not None, line["jp"] is not None])

val_data = test_data["train"].filter(filter_non_empty)
test_data = test_data["test"].filter(filter_non_empty)
train_data = train_data["train"].filter(filter_non_empty)


Loading dataset shards:   0%|          | 0/17 [00:00<?, ?it/s]

Filter:   0%|          | 0/2521388 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2521388 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20171104 [00:00<?, ? examples/s]

Map:   0%|          | 0/20168079 [00:00<?, ? examples/s]

Map:   0%|          | 0/2521020 [00:00<?, ? examples/s]

Dataset({
    features: ['en', 'jp', 'text'],
    num_rows: 2521020
})

In [22]:
train_data = load_from_disk("./traintmp")

In [24]:
train_data

Dataset({
    features: ['en', 'jp', 'text'],
    num_rows: 2016807
})

In [26]:
2016807 * 80/100

1613445.6

In [ ]:
train_data_1, val_data_1, test_data_1 = train_data[:1210084], train_data[1210084:1613445], train_data[1613445:]

In [19]:
train_data_1

{'en': ['He will raise the mood to all coming guests and neighbors.',
  'The sound material is sound data of piano solos performed by SHIBUYA from the CD "ATAK015 for maria" (2009) recorded with great sophistication using DSD (Direct Stream Digital) Recording, whose resolution is 128 times greater than that of a CD.From this data the piano keystroke sections were completely cut, extracting only the echo components, which then underwent processing and programming.',
  'Module This quite simply lists the name of the source module which defines the directive.',
  "FRANKY'S COLA BAR Law's Happy Pumpkin Party (with Law's coaster) 800 yen (tax included) 10Law is celebrating his birthday on October 6th and his sweets is a plate with pumpkin pudding and pumpkin tart.",
  'After a grand start, the rally faces its first SS (Special Stage) from south of Ulaanbaatar.',
  'Now you know how to draw Masha and the bear with a pencil and can start painting them.',
  'There are just the final stages to 

In [20]:
def add_prompt(line):
  return {'text': "<s>[INST]Translate from Japanese to English:  " + line['jp'] + " [/INST] " + line['en'] + " </s>"}



train_data = train_data.map(add_prompt)
val_data = val_data.map(add_prompt)
test_data = test_data.map(add_prompt)

In [12]:
tmp_train = train_data.train_test_split(0.9, seed=0)["train"]

In [17]:
train_data = tmp_train

In [18]:
train_data

Dataset({
    features: ['en', 'jp', 'text'],
    num_rows: 2016807
})

In [ ]:
base_model = "mistralai/Mistral-7B-Instruct-v0.2"
new_model = "mistral_7b_JParacrawl_1"



bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= True,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        resume_download=True,
        cache_dir=cache_dir
)
model.config.use_cache = False 
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()



tokenizer = AutoTokenizer.from_pretrained(base_model, cache_dir=cache_dir)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)


training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=100,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    evaluation_strategy="steps",
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to=None,
    do_eval=True,
    save_total_limit=5,
    logging_dir="./results/" + str(new_model),
)


trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

trainer.train(resume_from_checkpoint=True)

In [ ]:
model.save_pretrained("./model/" + new_model)

tokenizer.save_pretrained("./model/" + new_model)

# MISTRAL 2nd METHOD - Doesn't work

In [ ]:
base_model = "mistralai/Mistral-7B-Instruct-v0.2"
new_model = "mistral_7b_JParacrawl_1"



bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= True,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        resume_download=True,
        cache_dir=cache_dir
)
model.config.use_cache = False 
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()


tokenizer = AutoTokenizer.from_pretrained(base_model, cache_dir=cache_dir)
tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

In [ ]:
class StoppingCriteriaSub(StoppingCriteria):
    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop.to("cuda") for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        last_token = input_ids[0][-1]
        for stop in self.stops:
            if tokenizer.decode(stop) == tokenizer.decode(last_token):
                return True
        return False
prefix = "<|im_start|>"
suffix = "<|im_end|>\n"
sys_format = prefix + "system\n" + "Translate from Japanese to English" + suffix

assistant_format = prefix + "assistant\n"


def translate(dataset):
    user_format = prefix + "user\n" + dataset[1]["jp"]  + suffix
    input_text = sys_format + user_format + assistant_format
    messages = [
    {"role": "user", "content": input_text},
    ]
    tokens = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
    stop_words = []
    stop_words_ids = [tokenizer(stop_word, return_tensors='pt', add_special_tokens=False)['input_ids'].squeeze() for stop_word in stop_words]
    stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])
    output = model.generate(
        input_ids=tokens,
        max_new_tokens=120,
        repetition_penalty=1.5,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        forced_bos_token_id=tokenizer.bos_token_id,
        stopping_criteria=stopping_criteria,
        do_sample=True,
    )
    result = tokenizer.batch_decode(output, skip_special_tokens=True)
    return result[0].splitlines()[-1][9:]

print(translate(val_data))